# Create Azure curated environment

At its simples, finetuning a model requires model weights, and a dataset. You configure hyperparameters related to the training job and need to monitor it as it progresses. You also want to enable optimizatioins to make your training faster. Although simple conceptually, several questions presents themselves when fine tuning a model:

1. Are you starting with a completion model, and fine tuning it further on completion data, such as in the case of Domain Adaptation?
2. Are you starting with a completion model, and fine tuning it further with instructions?
3. How will you tune the instructions to reflect your user base queries the most appropriately?
4. If continually fine tuning, how will you assess your models against catastrophic forgetting?
5. How can you optimize the training job to conclude the fastest?

In addition to conceptual questions, several other engineering ones present themselves.
1. Will you use LoRA, QLoRA, or any other form of optimization?
2. Is your job parallel in nature, and requires distribution?
3. Do you need to register your model in MLFlow?

To help ease the configuration and execution of finetuning jobs, several frameworks are being developed. One of these frameworks is [Axolotl](https://github.com/OpenAccess-AI-Collective/axolotl) - a tool to streamline the fine-tuning of various AI models, offering support for multiple configurations and architectures. Companies often find themselves running hundreds of finetuning experiments in parallel (on different internal datasets, with different hyperparameters, on different model architectures).

## Goal of this notebook

In this notebook, you will create a new [Azure curated environment](https://learn.microsoft.com/en-us/azure/machine-learning/resource-curated-environments?view=azureml-api-2) to run Axolotl on Azure.
The reason you build your curated environmented is to further reduce training and deployment latency: You will define the python packages and software settings around training and scoring.
More specifically, you will:

1. connect to an AzureML workspace using the Python SDK v2.
2. Build a new curated environment based on a `dockerfile` that has been provided, which is based on the latest ACPT images. You will do this because in future notebooks, you will experiment with [Nebula](https://learn.microsoft.com/en-us/azure/machine-learning/reference-checkpoint-performance-for-large-models?view=azureml-api-2&tabs=PYTORCH). `Nebula` is not available in the Pypi package index, and only ships with ACPT.
3. Install `azureml-mlflow` and `mlflow` and patch them to integrate Axolotl with AzureML MLFlow. The patch has been provided to you and is part of the Docker build process.

###  A | Connecting to your workspace

First of all, you will use the Azure ML Python SDK to connect to your AML workspace. If you are running this notebook on a compute instance running on Azure, you can use the `azureml_py310_sdkv2` to execute the cells below.

In [ ]:
import azureml.core
workspace = azureml.core.Workspace.from_config()
workspace

Now that you are connected to the correct workspace, Run the cells below to build your curated environment, which will be built based on the docker context, more specifically the contents of the file `src/docker_axolotl_acpt/dockerfile`. 
The latest ACPT (beta) mcr.microsoft.com/aifx/acpt/stable-ubuntu2004-cu118-py310-torch220 ships with:

- Ubuntu 20.04
- CUDA 11.8
- Pytorch 2.2.0
- DeepSpeed 0.13.4
- Nebula 0.16

The process will take approximately 12 minutes. During this time, you can see the progress of the build job in the portal by clicking on the link displayed beneath the Jupyter cell.

In [ ]:
environment_display_name = 'axolotl_acpt'
all_environments_in_my_workspace = azureml.core.Environment.list(workspace).keys()

print(f"Building {environment_display_name} environment")
environment = azureml.core.Environment.from_docker_build_context(
    name = environment_display_name,
    docker_build_context=azureml.core.environment.DockerBuildContext.from_local_directory(
        workspace=workspace,
        path='src/docker_axolotl_acpt',
        dockerfile_path='dockerfile'
    )
)
environment = environment.register(workspace).build(workspace).wait_for_completion()

environment = azureml.core.Environment.get(workspace=workspace, name=environment_display_name)

This concludes the first notebook - you should now have have a curated environment that includes Axolotl on ACPT